In [93]:
import numpy as np
import cv2

# open input video
cap = cv2.VideoCapture('TestVid3.mp4')

# Default resolutions of the frame are obtained.The default resolutions are system dependent.
# We convert the resolutions from float to integer.
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))

# create video writer object to create output video
out = cv2.VideoWriter('fgmask.avi',cv2.VideoWriter_fourcc('M','J','P','G'), 15, (frame_width,frame_height))

# create background subtractor
fgbg = cv2.createBackgroundSubtractorMOG2(detectShadows=True, history=100)

# kernel for morphological operations
kernel1 = np.ones((3,3),np.uint8)
kernel2 = np.ones((9,9),np.uint8)

threshold = frame_width * frame_height / 200
print(threshold)

while(1):
    # read the frame
    ret, frame = cap.read()
    
    if ret is True:

        # apply the mask to the blurred frame
        fgmask = fgbg.apply(frame)
        
        # remove shadow
        fgmask[fgmask != 255] = 0
        
        cv2.imshow("fg", fgmask)
        
        # apply blur to remove noise
#         blur = cv2.GaussianBlur(fgmask, (5,5), 0)
        
#         cv2.imshow("blur", blur)
        
        # remove small noise
        img = cv2.morphologyEx(fgmask, cv2.MORPH_OPEN, kernel1)
        img = cv2.morphologyEx(img, cv2.MORPH_OPEN, kernel1)
        
        
        img = cv2.dilate(img,kernel2,iterations = 1)
        
        ########### COOOL
        
        
        count, img = cv2.connectedComponents(img, connectivity=8)
        
        unique, count = np.unique(img, return_counts=True)
        for i in range(0, len(count)):
            if count[i] < threshold:
                img[img == unique[i]] = 0
                
        img[img != 0] = 255
        
        img = img.astype(np.uint8)
        
        ############## END COOOL
        
        cv2.imshow("coool", img)
        
        colorImg = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
        out.write(colorImg)
            
        k = cv2.waitKey(3) & 0xff
        if k == 27:
            break
    else:
        break
    
cap.release()
out.release()

cv2.destroyAllWindows()

2044.8


In [75]:
import numpy as np
import cv2

cap = cv2.VideoCapture('fgmask.avi')

# Check if camera opened successfully
if (cap.isOpened()== False): 
    print("Error opening video stream or file")

# Default resolutions of the frame are obtained.The default resolutions are system dependent.
# We convert the resolutions from float to integer.
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))

out = cv2.VideoWriter('debug.avi',cv2.VideoWriter_fourcc('M','J','P','G'), 15, (frame_width,frame_height))

# stores all labelled frames with tube labels
volume = []
labelList = []
# number of labels in volume (aka number of tubes)
tubeLabel = 0

# count of OpenCV labels in previous frame
oldCount = 1

imgPrev = np.zeros((frame_width, frame_height),dtype=np.uint8)
imgCurr = np.zeros((frame_width, frame_height),dtype=np.uint8)

while(cap.isOpened()):
        
    # read the frame
    ret, frame = cap.read()
    
    
    # frame is read properly
    if ret is True:
        
        # fix JPEG compression issues
        frame[frame>50] = 255
        frame[frame!=255] = 0
        
        # Find connected components in the current image
        count, labels = cv2.connectedComponents(cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY), connectivity=8)
        
        if count > 1:
            
            labels = labels.astype(np.uint8)
            
            label_hue = np.uint8(179*labels/np.max(labels))
            blank_ch = 255*np.ones_like(label_hue)
            labeled_img = cv2.merge([label_hue, blank_ch, blank_ch])
            labeled_img = cv2.cvtColor(labeled_img, cv2.COLOR_HSV2BGR)
            labeled_img[label_hue==0] = 0
            
            cv2.imshow('label', labeled_img)
            out.write(labeled_img)
        
        # We need to match the components from current frame to component in previous frame
        tempLabels = labels.copy()
        tempList = []
        # iterate through components in current frame
        for i in range(1, count):
            
            # initially, no match is found wrt previous frame
            match = 0
            
            # iterate through components in previous frame
            for j in range(1, oldCount):
                
                imgCurr = labels.copy()
                imgCurr[imgCurr!=i] = 0
                imgCurr[imgCurr==i] = 255
                
#                 cv2.imshow("curr", imgCurr)
                
                imgPrev = volume[-1].copy()
                imgPrev[imgPrev!=labelList[-1][j-1]] = 0
                imgPrev[imgPrev==labelList[-1][j-1]] = 255
                
#                 cv2.imshow("prev", imgPrev)
                
                # multiply masks to find intersections of components
                prod = imgPrev/255 * imgCurr/255
                
                # product is black if there is no intersection
                # sum all the pixels
                total = np.sum(prod)
                if total/(np.sum(imgPrev/255)) > 0.5:
                    match = labelList[-1][j-1]
                    break
                    
            # match found for the selected component
            if match is not 0:
                # renaming selected component with matched component label
                tempLabels[labels==i] = labelList[-1][j-1]
                tempList.append(labelList[-1][j-1])
                
#                 print("Match")
#                 print(tempList)
#                 print(np.unique(tempLabels, return_counts=True))
            else:
                # new component found
                print("found" + str(i))
                print("tubelabel" + str(tubeLabel))
#                 print(np.unique(tempLabels, return_counts=True))
                tubeLabel += 1
                tempLabels[labels==i] = tubeLabel
                print(np.unique(tempLabels, return_counts=True))
                tempList.append(tubeLabel)
                
                print(tempList)
            
        volume.append(tempLabels)
        labelList.append(tempList)
        oldCount = count
        
        k = cv2.waitKey(0) & 0xff
        if k == 27:
            break
        
    # frame is not read properly
    else:
        break
    

cap.release()
out.release()
cv2.destroyAllWindows()
print('done')

found1
tubelabel0
(array([0, 1], dtype=uint8), array([406848,   2112], dtype=int64))
[1]
done


In [40]:
print(np.shape(volume))

(305, 480, 852)


In [23]:
x = volume[350]
x[x!=0] = 255
cv2.imshow('vol', x)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [64]:
num

6

In [134]:
import numpy as np
import cv2

def binary_mask(img, val, replacement_val):
    """
    Create a binary mask of the image

    Args:
        img: An image to be masked
        val: The value in the image to be masked
        replacement_val: The high value to be used in the binary mask

    Returns: Nothing. Masking is performed in-place.
    """

    img[img != val] = 0 # Pixels not equal to the required value are made 0
    img[img == val] = replacement_val # Pixels equal to the required value are set to replacement value
    return

def open_video(filename):
    """
    Open a video file using VideoCapture object

    Args:
        filename: The name of the file to be opened
            Pass 0 to open webcam, or -1 to search for cameras

    Raises:
        IOError: If the file or stream cannot be opened

    Returns: VideoCapture object, frame width, frame hight
    """

    cap = cv2.VideoCapture(filename) # Init VideoCapture object

    frame_width = frame_height = 0

    # Check if camera opened successfully
    if cap.isOpened() is False:
        raise IOError('Error opening video stream or file.')
    else:
        # Default resolutions of the frame are obtained.
        # We convert the resolutions from float to integer.
        frame_width = int(cap.get(3))
        frame_height = int(cap.get(4))

    return cap, frame_width, frame_height

def intersecting_regions(cur_img, cur_count, prev_img, prev_labels,
                            intersection_threshold = 0.2):

    """
    Find the number of intersecting regions between the any region in the
    current image and any regions in the previous image

    Args:
        cur_img: The labelled image of the current frame returned by OpenCV
            Connected Components

        cur_count: Number of components in cur_img

        prev_img: The labelled image of the previous frame returned by OpenCV
            Connected Components

        prev_labels: The tube labels present in the prev_img

        intersection_threshold: Fraction of intersection between regions in
        previous and current image

    Returns: A dict mapping intersecting regions in current frame to regions in
        the previous frame
            Example:
                {
                    2 : [2, 3],
                    3 : [4]
                }
    """


    # Create a copy of the images to prevent overwriting the parameters
    prev_img = prev_img.copy()
    cur_img = cur_img.copy()

    match_dict = {} # Dictionary to store the intersecting regions

    # Stores whether any current region intersect with multiple previous regions
    multiple_intersections = False

    for region_cur in range(1, cur_count):

        # Create a copy of the image for this iteration
        cur_img_copy = cur_img.copy()
        binary_mask(cur_img_copy, region_cur, 1)

        matches = [] # Clear matches list for each region in current image

        for region_prev in prev_labels:

            # Create a copy of the image for this iteration
            prev_img_copy = prev_img.copy()
            binary_mask(prev_img_copy, region_prev, 1)

            # Multiply masks to find the intersections of regions
            intersection = prev_img_copy * cur_img_copy

            # Number of intersecting pixels
            intersection_count = np.sum(intersection)

            # Number of pixels in previous region
            region_prev_count = np.sum(prev_img_copy)

            # Region is matched if intersection fraction greater than threshold
            if intersection_count/region_prev_count > intersection_threshold:
                matches.append(region_prev)

        if len(matches) > 1:
            multiple_intersections = True

        match_dict[region_cur] = matches

    return multiple_intersections, match_dict



try:
    cap, frame_width, frame_height = open_video('fgmask.avi')
except IOError as error:
    print(error)
    print('Check the filename or camera.')
    
volume = [np.zeros((frame_height, frame_width), dtype=np.uint8)] # 3D volume; A list for storing the labelled images
labels = [[1]] # A list storing the tube labels in each image in the volume
tube_num = 0 # Number of tubes found
prev_count = 1 # OpenCV returns at least 1 label (i.e. the background)

overwrite_dict = {}

out = cv2.VideoWriter('debug.avi', cv2.VideoWriter_fourcc('M', 'J', 'P', 'G'),
                        15, (frame_width,frame_height))

while(cap.isOpened()):
    ret, frame = cap.read() # Read the frame

    if ret is False: # Frame is read properly
        break

    # Fix JPEG compression issues
    frame[frame>50] = 255
    frame[frame!=255] = 0

    # Find connected components in the current image
    cur_count, cur_img = cv2.connectedComponents(cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY), connectivity=8)

    ###############################################################
    # Debug output of connected components
    if cur_count > 1:

        cur_img = cur_img.astype(np.uint8)

        label_hue = np.uint8(179*cur_img/np.max(cur_img))
        blank_ch = 255*np.ones_like(label_hue)
        labeled_img = cv2.merge([label_hue, blank_ch, blank_ch])
        labeled_img = cv2.cvtColor(labeled_img, cv2.COLOR_HSV2BGR)
        labeled_img[label_hue==0] = 0

        cv2.imshow('label', labeled_img)
        out.write(labeled_img)
    ###############################################################

    ret, match_dict = intersecting_regions(cur_img, cur_count, volume[-1], labels[-1])

    cur_label = []

    cur_img_output = cur_img.copy()

    if ret is True:
        # Some current region is intersecting with multiple previous regions

        cur_img_copy = cur_img.copy()

        # Erode the image to try and separate the regions
        kernel = np.ones((5,5), np.uint8)
        cur_img_copy = cv2.erode(cur_img_copy, kernel, iterations = 1)

        cur_count_copy, cur_img_copy = cv2.connectedComponents(cur_img_copy, connectivity=8)

        # Find the number of intersections using eroded image
        ret, match_dict_copy = intersecting_regions(cur_img_copy, cur_count_copy, volume[-1], labels[-1])


        print("intersection " + str(ret) + str(match_dict_copy))

        if ret is False:
            print("erosion success")
            # The intersecting regions have been separated by erosion
            # Replace the image by the eroded image
            cur_img = cur_img_copy
            cur_count = cur_count_copy
            match_dict = match_dict_copy
        else:
            # The intersecting regions did not separate
            # Overwrite matching labels in the volume with current label

            tube_num += 1

            for region in match_dict:
                if len(match_dict[region]) > 1:

                    for index in match_dict[region]:
                        for frame in volume:
                            # Update labels of tubes in previous frame
#                             frame[frame == index] = tube_num
                            # Add an entry to the overwrite dict
                            overwrite_dict[index] = [tube_num]

                    # Update the dict
                    match_dict[region] = [tube_num]
#                     cur_img[cur_img == region] = tube_num
#                     cur_label.append(tube_num)

                    


    # There are no intersection issues; Proceed normally
    for region in match_dict:
        if len(match_dict[region]) == 0:
            # New region found since there is no match to any previous region
            tube_num += 1
            cur_img_output[cur_img == region] = tube_num
            cur_label.append(tube_num)

        else:
            # Some previous region was found, relabel the region
            cur_img_output[cur_img == region] = match_dict[region][0]
            cur_label.append(match_dict[region][0])
    

    volume.append(cur_img_output)
#     print(cur_label)
    labels.append(cur_label) # Convert keys in the match_dict to a list of labels of current image
    prev_count = cur_count

    k = cv2.waitKey(1) & 0xff
    if k == 27:
        break

# Update previous frames
# print(overwrite_dict)
for index in overwrite_dict:
    for frame in volume:
        # Update labels of tubes in previous frame
        frame[frame == index] = tube_num
    
cap.release()
out.release()
cv2.destroyAllWindows()
print('done')


intersection True{1: [1], 2: [2, 2], 3: [2, 2]}
intersection True{1: [1], 2: [3, 3], 3: [3, 3]}
intersection True{1: [1], 2: [4, 4], 3: [4, 4]}
intersection True{1: [1], 2: [5, 5], 3: [5, 5]}
intersection True{1: [1], 2: [6, 6]}
intersection False{1: [1], 2: [7]}
erosion success
intersection True{1: [1, 7]}
intersection True{1: [8, 8], 2: [8, 8]}
intersection True{1: [9, 9], 2: [9, 9]}
intersection True{1: [10, 10], 2: [10, 10]}
intersection True{1: [11, 11], 2: [11, 11]}
intersection True{1: [12, 12], 2: [12, 12]}
intersection True{1: [13, 13], 2: [13, 13]}
intersection True{1: [14, 14], 2: [14, 14]}
intersection True{1: [15, 15], 2: [15, 15]}
intersection True{1: [16, 16], 2: [16, 16]}
intersection True{1: [17, 17], 2: [17, 17]}
intersection True{1: [18, 18], 2: [18, 18]}
intersection True{1: [19, 19]}
{2: [3], 3: [4], 4: [5], 5: [6], 6: [7], 1: [8], 7: [8], 8: [9], 9: [10], 10: [11], 11: [12], 12: [13], 13: [14], 14: [15], 15: [16], 16: [17], 17: [18], 18: [19], 19: [20]}
done


In [128]:
tube_num

20

In [135]:
uniq, __ = np.unique(volume, return_counts=True)
print(uniq)
print(__)

[ 0 20]
[123857086   1284674]


In [129]:
for i in range(0, len(uniq)):
    
    out = cv2.VideoWriter('maskTube{0}.avi'.format(i),cv2.VideoWriter_fourcc('M','J','P','G'), 15, (frame_width,frame_height))
    
    for frame in volume:
        frameCopy = frame.copy()
        frameCopy[frameCopy==uniq[i]] = 255
        frameCopy[frameCopy!=255] = 0
        frameCopy = frameCopy.astype(np.uint8)
        out.write(cv2.cvtColor(frameCopy, cv2.COLOR_GRAY2BGR))

    out.release()